# Cedric API Pipeline for Patient Health

This notebook uses patient health data to generate personalized health recommendations through the Cedric API.

In [ ]:
import requests
import json
import pandas as pd
from typing import Dict, Any, Optional, List

In [ ]:
class CedricPipeline:
    """Pipeline for making API calls to Cedric"""
    
    def __init__(self, api_url: str, api_key: Optional[str] = None, timeout: int = 30):
        self.api_url = api_url
        self.api_key = api_key
        self.timeout = timeout
        self.headers = {
            'Content-Type': 'application/json'
        }
        
        if api_key:
            self.headers['Authorization'] = f'Bearer {api_key}'
    
    def process_text(self, text: str, **kwargs) -> str:
        """Process text through the Cedric API"""
        payload = {
            'text': text,
            **kwargs
        }
        
        try:
            response = requests.post(
                self.api_url,
                headers=self.headers,
                data=json.dumps(payload),
                timeout=self.timeout
            )
            response.raise_for_status()
            
            result = response.json()
            return result.get('text', '')
            
        except requests.exceptions.RequestException as e:
            raise Exception(f"Error making API call to Cedric: {str(e)}")

In [ ]:
# Configure the pipeline
CEDRIC_API_URL = "https://api.cedric.example.com/process"  # Replace with actual API URL
CEDRIC_API_KEY = "your_api_key_here"  # Replace with your API key

# Create the pipeline
pipeline = CedricPipeline(api_url=CEDRIC_API_URL, api_key=CEDRIC_API_KEY)

In [ ]:
# Load patient data
def load_patient_data():
    # Load demographic data
    with open('data/patient_demographic.json', 'r') as f:
        demographic = json.load(f)
    
    # Load health profile
    with open('data/patient_health.json', 'r') as f:
        health = json.load(f)
    
    # Load visit information
    with open('data/patient_visits.json', 'r') as f:
        visits = json.load(f)
    
    # Load vital signs analysis
    with open('data/vital_signs_analysis.json', 'r') as f:
        vital_analysis = json.load(f)
    
    # Load raw vital signs data
    vital_signs = pd.read_csv('data/patient_vital_signs.csv')
    
    # Load food log data
    with open('data/daily_food_log.json', 'r') as f:
        food_log = json.load(f)
    
    return {
        'demographic': demographic,
        'health': health,
        'visits': visits,
        'vital_analysis': vital_analysis,
        'vital_signs': vital_signs,
        'food_log': food_log
    }

## Function 1: Generate Health Plan

In [ ]:
def create_health_plan_prompt(patient_data):
    """Create a prompt for generating a health plan"""
    # Read the base prompt
    with open('cedric_health_prompt.txt', 'r') as f:
        base_prompt = f.read()
    
    # Extract key patient information
    patient = patient_data['demographic']['patient']
    health_profile = patient_data['health']['patientHealthProfile']
    vital_analysis = patient_data['vital_analysis']['vitalSignsAnalysis']
    
    # Create a patient summary
    patient_summary = f"""Patient Summary:
Name: {patient['firstName']} {patient['lastName']}
Age: 30 (DOB: {patient['dateOfBirth']})
Sex: {patient['sex']}
Height: {patient['height']['value']} {patient['height']['unit']}
Weight: {patient['weight']['value']} {patient['weight']['unit']}

Health Conditions:
- {health_profile['healthConditions']['chronicConditions'][0]['condition']} (Status: {health_profile['healthConditions']['chronicConditions'][0]['status']})
- {health_profile['healthConditions']['chronicConditions'][1]['condition']} (Status: {health_profile['healthConditions']['chronicConditions'][1]['status']})

Current Medications:
- {health_profile['medications']['currentMedications'][0]['name']} {health_profile['medications']['currentMedications'][0]['dosage']} ({health_profile['medications']['currentMedications'][0]['purpose']})
- {health_profile['medications']['currentMedications'][1]['name']} {health_profile['medications']['currentMedications'][1]['dosage']} ({health_profile['medications']['currentMedications'][1]['purpose']})

Allergies:
- {health_profile['allergies'][0]['name']} (Severity: {health_profile['allergies'][0]['severity']})
- {health_profile['allergies'][1]['name']} (Severity: {health_profile['allergies'][1]['severity']})

Vital Signs Concerns:
- High Blood Pressure at times: {', '.join([reading['time'] for reading in vital_analysis['abnormalReadings']['highBloodPressure']])}
- High Blood Sugar at times: {', '.join([reading['time'] for reading in vital_analysis['abnormalReadings']['highBloodSugar']])}

Doctor's Notes:
{patient_data['visits']['patientVisits']['visits'][0]['notes']}
"""
    
    # Combine the base prompt with the patient summary
    full_prompt = f"{patient_summary}\n\n{base_prompt}"
    return full_prompt

In [ ]:
def generate_health_plan():
    """Generate a personalized health plan using Cedric API"""
    # Load patient data
    patient_data = load_patient_data()
    
    # Create prompt
    prompt = create_health_plan_prompt(patient_data)
    
    # Process through Cedric
    print("Generating health plan...")
    response = pipeline.process_text(prompt)
    
    # Save the response to a file
    with open('cedric_health_plan.txt', 'w') as f:
        f.write(response)
    
    print("Health plan generated and saved to 'cedric_health_plan.txt'")
    return response

## Function 2: Analyze Food Intake

In [ ]:
def create_diet_analysis_prompt(patient_data):
    """Create a prompt for analyzing food intake and allergies"""
    # Read the base prompt
    with open('cedric_diet_prompt.txt', 'r') as f:
        base_prompt = f.read()
    
    # Extract key patient information
    patient = patient_data['demographic']['patient']
    health_profile = patient_data['health']['patientHealthProfile']
    food_log = patient_data['food_log']['dailyFoodLog']
    
    # Format allergies information
    allergies = [f"{allergy['name']} (Type: {allergy['type']}, Severity: {allergy['severity']}, Reaction: {allergy['reaction']})" 
                for allergy in health_profile['allergies']]
    allergies_text = "\n- ".join([""] + allergies)
    
    # Format health conditions
    conditions = [f"{condition['condition']} (Status: {condition['status']})" 
                 for condition in health_profile['healthConditions']['chronicConditions']]
    conditions_text = "\n- ".join([""] + conditions)
    
    # Format medications with instructions
    medications = [f"{med['name']} {med['dosage']} ({med['purpose']}) - Instructions: {med['instructions']}" 
                  for med in health_profile['medications']['currentMedications']]
    medications_text = "\n- ".join([""] + medications)
    
    # Format food log
    food_log_text = f"Date: {food_log['date']}\n\n"
    
    for meal_name, meal_data in food_log['meals'].items():
        food_log_text += f"{meal_name.capitalize()} (Time: {meal_data['time']}):\n"
        for item in meal_data['items']:
            food_log_text += f"- {item['food']}, {item['portion']}"
            if 'nutrition' in item:
                nutrition = item['nutrition']
                food_log_text += f", {nutrition['calories']} calories, {nutrition['protein']}g protein, {nutrition['fat']}g fat, {nutrition['sugar']}g sugar, {nutrition['salt']}g salt"
            if 'components' in item:
                food_log_text += f" (contains: {', '.join(item['components'])})"
            food_log_text += "\n"
        food_log_text += "\n"
    
    if 'dailyTotalsSoFar' in food_log:
        totals = food_log['dailyTotalsSoFar']
        food_log_text += f"Daily totals so far:\n"
        food_log_text += f"- Calories: {totals['calories']}\n"
        food_log_text += f"- Protein: {totals['protein']}g\n"
        food_log_text += f"- Fat: {totals['fat']}g\n"
        food_log_text += f"- Sugar: {totals['sugar']}g\n"
        food_log_text += f"- Salt: {totals['salt']}g"
    
    # Create a patient diet summary
    patient_diet_summary = f"""Patient Diet Analysis:
Name: {patient['firstName']} {patient['lastName']}
Age: 30 (DOB: {patient['dateOfBirth']})
Sex: {patient['sex']}
Height: {patient['height']['value']} {patient['height']['unit']}
Weight: {patient['weight']['value']} {patient['weight']['unit']}

Known Allergies:{allergies_text}

Health Conditions:{conditions_text}

Current Medications:{medications_text}

Today's Food Log:
{food_log_text}
"""
    
    # Combine the base prompt with the patient diet summary
    full_prompt = f"{patient_diet_summary}\n\n{base_prompt}"
    return full_prompt

In [ ]:
def analyze_food_intake():
    """Analyze patient's food intake considering allergies and health conditions"""
    # Load patient data
    patient_data = load_patient_data()
    
    # Create prompt
    prompt = create_diet_analysis_prompt(patient_data)
    
    # Process through Cedric
    print("Analyzing food intake...")
    response = pipeline.process_text(prompt)
    
    # Save the response to a file
    with open('cedric_diet_recommendations.txt', 'w') as f:
        f.write(response)
    
    print("Diet analysis completed and saved to 'cedric_diet_recommendations.txt'")
    return response

## Function 3: Generate JSON Diet Plan

In [ ]:
def create_diet_plan_prompt(patient_data):
    """Create a prompt for generating a structured diet plan in JSON format"""
    # Read the base prompt
    with open('cedric_diet_plan_prompt.txt', 'r') as f:
        base_prompt = f.read()
    
    # Extract key patient information
    patient = patient_data['demographic']['patient']
    health_profile = patient_data['health']['patientHealthProfile']
    vital_analysis = patient_data['vital_analysis']['vitalSignsAnalysis']
    
    # Format allergies information
    allergies = [f"{allergy['name']} (Type: {allergy['type']}, Severity: {allergy['severity']}, Reaction: {allergy['reaction']})" 
                for allergy in health_profile['allergies']]
    allergies_text = "\n- ".join([""] + allergies)
    
    # Format health conditions
    conditions = [f"{condition['condition']} (Status: {condition['status']})" 
                 for condition in health_profile['healthConditions']['chronicConditions']]
    conditions_text = "\n- ".join([""] + conditions)
    
    # Format medications with instructions
    medications = [f"{med['name']} {med['dosage']} ({med['purpose']}) - Instructions: {med['instructions']}" 
                  for med in health_profile['medications']['currentMedications']]
    medications_text = "\n- ".join([""] + medications)
    
    # Create a patient summary
    patient_summary = f"""Patient Summary for Diet Plan:
Name: {patient['firstName']} {patient['lastName']}
Age: 30 (DOB: {patient['dateOfBirth']})
Sex: {patient['sex']}
Height: {patient['height']['value']} {patient['height']['unit']}
Weight: {patient['weight']['value']} {patient['weight']['unit']}

Known Allergies:{allergies_text}

Health Conditions:{conditions_text}

Current Medications:{medications_text}

Vital Signs Concerns:
- High Blood Pressure at times: {', '.join([reading['time'] for reading in vital_analysis['abnormalReadings']['highBloodPressure']])}
- High Blood Sugar at times: {', '.join([reading['time'] for reading in vital_analysis['abnormalReadings']['highBloodSugar']])}

Doctor's Notes:
{patient_data['visits']['patientVisits']['visits'][0]['notes']}
"""
    
    # Combine the base prompt with the patient summary
    full_prompt = f"{patient_summary}\n\n{base_prompt}"
    return full_prompt

In [ ]:
def generate_json_diet_plan():
    """Generate a structured diet plan in JSON format using Cedric API"""
    # Load patient data
    patient_data = load_patient_data()
    
    # Create prompt
    prompt = create_diet_plan_prompt(patient_data)
    
    # Process through Cedric
    print("Generating JSON diet plan...")
    response = pipeline.process_text(prompt)
    
    # Parse the response as JSON
    try:
        diet_plan = json.loads(response)
        
        # Save the response to a JSON file
        with open('cedric_diet_plan.json', 'w') as f:
            json.dump(diet_plan, f, indent=2)
        
        print("Diet plan generated and saved to 'cedric_diet_plan.json'")
        return diet_plan
    except json.JSONDecodeError:
        print("Warning: Response is not valid JSON. Saving as text instead.")
        with open('cedric_diet_plan.txt', 'w') as f:
            f.write(response)
        print("Diet plan saved as text to 'cedric_diet_plan.txt'")
        return response

## Run the Functions

In [ ]:
# Generate health plan
health_plan = generate_health_plan()

# Display the results
print("\nCedric's Health Plan Recommendations:")
print("=====================================")
print(health_plan)

In [ ]:
# Analyze food intake
diet_recommendations = analyze_food_intake()

# Display the results
print("\nCedric's Diet Recommendations:")
print("=====================================")
print(diet_recommendations)

In [ ]:
# Generate JSON diet plan
diet_plan = generate_json_diet_plan()

# Display a summary of the diet plan
if isinstance(diet_plan, dict):
    print("\nDiet Plan Summary:")
    print("=====================================")
    print(f"Patient: {diet_plan.get('dietPlan', {}).get('patientName', 'Unknown')}")
    print(f"Daily Calorie Target: {diet_plan.get('dietPlan', {}).get('dailyTargets', {}).get('calories', {}).get('target', 'Unknown')} kcal")
    print(f"Plan Duration: {diet_plan.get('dietPlan', {}).get('validityPeriod', 'Unknown')}")
    print(f"Number of Days: {len(diet_plan.get('dietPlan', {}).get('weeklyPlan', []))}")
    
    # Display first day's meals as example
    if diet_plan.get('dietPlan', {}).get('weeklyPlan'):
        day1 = diet_plan['dietPlan']['weeklyPlan'][0]
        print(f"\nSample Day (Day {day1.get('day', 1)}) Meals:")
        for meal_name, meal_data in day1.get('meals', {}).items():
            print(f"- {meal_name.capitalize()} ({meal_data.get('time', 'N/A')}): {', '.join([item.get('food', 'Unknown') for item in meal_data.get('items', [])])}")
else:
    print("\nDiet Plan (text format):")
    print("=====================================")
    print(diet_plan[:500] + "..." if len(diet_plan) > 500 else diet_plan)

## Sample Usage with Function Calls

In [ ]:
def get_cedric_recommendations(query_type):
    """Get recommendations from Cedric based on query type"""
    if query_type.lower() == "health plan":
        return generate_health_plan()
    elif query_type.lower() == "diet analysis":
        return analyze_food_intake()
    elif query_type.lower() == "json diet plan":
        return generate_json_diet_plan()
    else:
        return "Invalid query type. Please use 'health plan', 'diet analysis', or 'json diet plan'."

In [ ]:
# Example function call
query = "json diet plan"  # or "health plan" or "diet analysis"
recommendations = get_cedric_recommendations(query)
print(f"\nRecommendations for {query} generated successfully.")